# Maximum Collinear Points
Given a set of points in a two-dimensional plane, determine the maximum number of points that lie along the same straight line.

```python
Input: points = [[1, 1], [1, 3], [2, 2], [3, 1], [3, 3], [4, 4]]
Output: 4
```

**Constraints:**
- The input won't contain duplicate points.

## Intuition

Two or more points are **collinear** if they lie on the same straight line. In other words, the **slope** between any pair of points among them will be equal.

The slope between two points (X_a, Y_a) and (X_b, Y_b) is calculated as:

$$
\text{slope} = \frac{Y_b - Y_a}{X_b - X_a}
$$

Using this formula, we can compute the slope between all pairs of points. However, simply counting how many pairs share the same slope is **not sufficient**, since multiple pairs may have the same slope but not lie on the same line.

Instead, we can improve the approach by fixing one point at a time as a **focal point**, and then counting how many other points are collinear with it.

---

### Improved Strategy

1. For each point in the input, treat it as a **focal point**.
2. For every other point, compute the **slope** with respect to the focal point.
3. Use a hash map (dictionary) to count how many times each slope occurs.
4. The maximum value in this hash map gives the number of points that are collinear with the current focal point.
5. Repeat for each point, and keep track of the overall maximum.

---

### Edge Case: Vertical Lines

If two points have the same x-coordinate, the denominator of the slope formula becomes 0. Since division by zero is undefined, we handle vertical lines by using a special representation such as `'inf'` or a constant symbol to denote **infinite slope**.

---

### Avoiding Precision Issues

Floating-point division may lead to precision errors. To avoid this, represent slopes as **fractions of integers**, i.e., a tuple (dy, dx), where:

- \(dy = Y_b - Y_a\)
- \(dx = X_b - X_a\)

This preserves exact values.

---

### Consistent Slope Representation

To ensure that different pairs with the same slope have identical representations, reduce the slope to its **simplest form** by dividing both components by their **greatest common divisor (GCD)**.

For example, a slope of (4, 6) should be reduced to (2, 3). Also, always normalize the sign: typically, keep the denominator positive.

---

### Greatest Common Divisor (GCD)

The GCD of two integers is the largest number that divides both without leaving a remainder. By dividing \(dy\) and \(dx\) by their GCD, we get a normalized and consistent representation of the slope.

This ensures that all equivalent slopes are stored using the same key in the hash map, leading to correct counts.

In [1]:
from typing import List, Tuple
from collections import defaultdict
import math


def maximum_collinear_points(points: List[List[int]]) -> int:
    res = 0

    for i in range(len(points)):
        res = max(res, max_points_from_focal_point(i, points))
    
    return res


def max_points_from_focal_point(focal_point_index: int, points: List[List[int]]) -> int:
    slopes_map = defaultdict(int)
    max_points = 0

    for j in range(len(points)):
        
        if j != focal_point_index:
            curr_slope = get_slope(points[focal_point_index], points[j])
            slopes_map[curr_slope] += 1
            max_points = max(max_points, slopes_map[curr_slope])
    
    return max_points + 1


def get_slope(p1: List[int], p2: List[int]) -> Tuple[int, int]:
    rise = p2[1] - p1[1]
    run = p2[0] - p1[0]

    if run == 0:
        return (1, 0)

    gcd_val = math.gcd(rise, run)

    return (rise // gcd_val, run // gcd_val)

### Complexity Analysis

The **time complexity** is O(n^2 \log m), where:
- n is the number of points,
- m is the maximum absolute value among all coordinates.

#### Breakdown:
- The `gcd(rise, run)` function has a time complexity of O(log(min(rise, run))), which is at most O(log m) assuming coordinates are bounded by m.
- For each focal point, we compute the slope to every other point, invoking `gcd` n - 1 times. This gives O(nlog m) per focal point.
- Since we do this for all n points, the total time complexity is:


$$O(n \cdot (n \log m)) = O(n^2 \log m)$$


---

The **space complexity** is O(n):

- For each focal point, we use a hash map to count the occurrences of each unique slope.
- In the worst case, this map contains up to n - 1 entries (one for each other point).

Hence, the space complexity per focal point is O(n), and since we reuse the map per iteration, the overall space usage remains O(n).
